# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

This is a **Classification** problem.
Because the problem output should be either 'will graduate' or 'will dropout', which is a discrete value output, thus it is a classification problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [17]:
# TODO: Calculate number of students
n_students = student_data.shape[0] # number of rows

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1 # number of columns, minus the output 'passed' column

# TODO: Calculate passing students
n_passed = np.sum(student_data['passed']=='yes') # sum up passed students

# TODO: Calculate failing students
n_failed = np.sum(student_data['passed']=='no') # sum up the failed students

# TODO: Calculate graduation rate
grad_rate = float(n_passed) / n_students # convert to float division to output a percentage

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 0.67%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [18]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [19]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [24]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=0)
# picked random_state = 0

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

I picked Gaussian Naive Bayes(GaussianNB), K-Nearest Neighbors(KNeighbors), Support Vector Machines(SVM).


**I refered to <a href="https://en.wikipedia.org/wiki/Naive_Bayes_classifier">Naive Bayes classifier</a>, <a href="https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm">k-nearest neighbors algorithm</a>, <a href="http://www.sciencedirect.com/science/article/pii/S2213158213001332">a paper in neurological studies</a>, <a href="https://en.wikipedia.org/wiki/Support_vector_machine">Support vector machine</a> to answer this question.**

1. GaussianNB
    - Application: It could be used for text categorization, for example spam email detection, where we are given bags of words as features and try to predict if it's a spam.
    - Pros: It is rather efficient and have worked well in real applications. It works well when the assumption of conditional indepence is met among the features.
    - Cons: NB makes the assumption that given the class label, the features are independent of each other, which might not be the case for certain problems. Also comparisons have shown that NB may be outperformed by other approaches, such as boosted trees or random forests.
    - Reason: After a first look of the problem, it seems very similar to a spam detection problem, with an implicit assumption that "just like the word 'deal' points to a spam, a high absence number points to a school dropout". In video lectures I've learnt that NB performs well on spam detection in real world applications and thought of it as a first choice.
    
    
2. KNeighbors
    - Application: kNN is a very versatile and could be used in many different classification and regression problems. For example, <a href="http://www.sciencedirect.com/science/article/pii/S2213158213001332">this paper</a> showed an application using kNN in neuraological studies to segment matter lesion.
    - Pros: It's a versatile and "simple" method. Being a non-parametric instance-based learning algorithm, kNN performs well when there's well-defined local structures in the data.
    - Cons: It may "overbelieve" local data, and may perform badly on very skewed data. For example in binary classification, if the two classes are very "muddily" mixed together, kNN may result in wierd looking decision boundaries.
    - Reason: An intuition is that maybe passed students are similar, and dropouts are alike. Implicitly, that's saying the local strutures in the feature space may be interesting to look at. Since kNN is a powerful yet simple tool to adapt locally, this became my second choice.
    
    
3. SVM
    - Application: SVM is a popular and powerful algorithm in both classification and regression problems. <a href="Support vector machine">The wikipedia page</a> mentioned applications such as text categorization and hand-written digits recognition.
    - Pros: Known as a "large margin" classifier, SVM tends to seperate the feature space (or a higher dimension projected space) with the largest margin. With kernel tricks, SVM is very useful in applications where a "good" decision boundary is desired. SVM performs well when there exist a "good" decision boundary, i.e. with structures that's easily seperable.
    - Cons: SVM tries to seperate the space with a large margin. Thus if the data points are very sticky and are mixed up together (take white noise for an example), SVM may not perform as well. Also, in kernel tricks, SVM tries to seperate in some higher dimensional space, this may be computationally costly (personal opinion).
    - Reason: A student intervention system should only output negative results when it's quite confident that the student is not doing well. Otherwise it may be bad for day-to-day school management and students' psychological health. Therefore we desire a classifier that gives reasonable decision boundaries. Since SVM has the property to fit to the "largest margin", this is my third choice.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [25]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [38]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn.naive_bayes import GaussianNB
# from sklearn import model_B
from sklearn.neighbors import KNeighborsClassifier
# from skearln import model_C
from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = KNeighborsClassifier()
clf_C = SVC()

# TODO: Set up the training set sizes
# since it's not explicitly stated how the training points should be drawn from X_train and y_train,
# I decided just to take the first 100/200/300 rows in the set
X_train_100 = X_train[0:100]
y_train_100 = y_train[0:100]

X_train_200 = X_train[0:200]
y_train_200 = y_train[0:200]

X_train_300 = X_train[0:300]
y_train_300 = y_train[0:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0014 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8550.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7481.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0015 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 0.8321.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.7132.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0018 seconds
Made predictions in 0.0008 seconds.
F1 score for training set: 0.8088.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.7500.
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0009 seconds
Made predictions in 0.0022 seconds.
F1 score for training set: 0.7972.
Made predictions in 0.0025 seconds.
F1 score for test set: 0.7068.
Training a KNeighborsClassifier using a training set size of 200. . .


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0014s         |         0.0003s        |     0.8500       |      0.7481     |
| 200               |         0.0015s         |         0.0006s        |     0.8321       |      0.7132     |
| 300               |         0.0018s         |         0.0006s        |     0.8088       |      0.7500     |

** Classifer 2 - KNeighborsClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0009s         |         0.0025s        |     0.7972       |      0.7068     |
| 200               |         0.0012s         |         0.0017s        |     0.8571       |      0.7172     |
| 300               |         0.0009s         |         0.0026s        |     0.8722       |      0.7482     |

** Classifer 3 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0014s         |         0.0008s        |     0.8591       |      0.7838     |
| 200               |         0.0039s         |         0.0014s        |     0.8693       |      0.7755     |
| 300               |         0.0082s         |         0.0019s        |     0.8692       |      0.7586     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

I decided to pick SVM model.

**Experiment Results**
- Training Time: GaussianNB has an intermediate training time, in the order of 0.001s, growing rather slowly with the size of training set. KNN has slightly shorter training time than GaussianNB, also not clearly dependent on the size of training set. SVM has the longest training time, growing rather fast with the size of training test. 
- Prediction Time (test): Both GaussianNB and SVM gives predictions on the test set relatively fast, in the order of 0.001s. Since KNN is a instance-based learning algorithm, the prediction time is longer than the other two models.
- F1 score (train): GaussianNb gives intermediate F1 score on the training set. But as the size of training set grows, the F1 score goes down, which might indicate that the model is suffering from high bias, not even fitting well to the training data. KNN gives relatively good F1 score, especially with more training examples. SVM does very well in terms of F1 score, performing consistently good.
- F1 score (test): In the first 6 rows in the tables, neither GaussianNb nor KNN gave a F1 score higher than 0.7500, considering that their performances on test set are better, this may suggest that both models are not generalizing well to the test set. On the other hand, SVM gives comparatively better results on the test set, exceeding 0.7500 in three runs. Although it's decreasing with the growth of training set size, still it outperforms the other two models

**Other Considerations**
- Data: In a supervised learning setup, our original dataset only has 395 examples, personally I think that's a very small amount of data. Thus, an algorithm that overfits the data may be very bad in real application.
- Resources and Cost: All 3 of the models run quite fast. In a system where we need to monitor and maybe intervene a student's academic life, a time that's in the order of 0.001s is neglectable. Also, it makes sense to put more penalties on prediction time than training time, since we want to engage as soon as a student shows enough evidence to possibly drop out, and potentially influence his future. On the other hand, a longer system setup with fast predictions doesn't seem so bad.
- Performance: This is the number 1 consideration we should have. Compared with a slower but accurate system, a fast but inaacurate system maybe disastrous for the student and the family (No parents want to hear that their kid is going to drop out, right?). An accurate prediction is what makes this system make sense in the first place.

In all, although SVM takes longer to train, it makes predictions rather fast, and most importantly, tends to give the best predictions on the test set. Thus, it is my choice for the model.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

What is the reason that makes a student drop out?

Jobs of his/her parents? Travel time vs. study time? Absences? Or a combination of these together with some other factors? A student with a poorer family may just invest a lot of time studying to improve his life, a student with many absences may just be ill and spend more time to catch up. A good model should not only try to differentiate graduates and dropouts with great confidence, but also consider all factors at the same time.

SVM does exactly that. It not only tries to seperate graduates and dropouts with the a big difference, but also consider all factors, and combinations of factors. It seems to be a suitable model to choose for our system, because it gives good accuracy and also predicts quite fast. Both of which are essential to help a student to get on track towards an academic success.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [48]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
# TODO: Create the parameters list you wish to tune

# I referred to http://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html
# which gives an example on how to tune the C and gamma parameters in SVM using rbf kernel

C_range = np.logspace(-2, 10, 13)
# >>> np.logspace(-2,10,13)
# array([  1.00000000e-02,   1.00000000e-01,   1.00000000e+00,
#          1.00000000e+01,   1.00000000e+02,   1.00000000e+03,
#          1.00000000e+04,   1.00000000e+05,   1.00000000e+06,
#          1.00000000e+07,   1.00000000e+08,   1.00000000e+09,
#          1.00000000e+10])

gamma_range = np.logspace(-9, 3, 13)
# >>> np.logspace(-9,3,13)
# array([  1.00000000e-09,   1.00000000e-08,   1.00000000e-07,
#          1.00000000e-06,   1.00000000e-05,   1.00000000e-04,
#          1.00000000e-03,   1.00000000e-02,   1.00000000e-01,
#          1.00000000e+00,   1.00000000e+01,   1.00000000e+02,
#          1.00000000e+03])

parameters = dict(gamma=gamma_range, C=C_range)

# TODO: Initialize the classifier
clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label=1)
# F1 = 2 * (precision * recall) / (precision + recall)
# since in our case there's only one binary class to predict, which is "passed"
# I left the "average" parameter for f1_score to be default, as None
# setting pos_label to 1 or 0 would actually result in the same score here
# because predicting passed students correctly is essentially the same thing as predicting dropouts correctly
# just to explicitly examine that, I set pos_label to 0 and 1, and got the same results

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0050 seconds.
Tuned model has a training F1 score of 0.8434.
Made predictions in 0.0019 seconds.
Tuned model has a testing F1 score of 0.7862.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final model's F1 score for training is 0.8434, for testing is 0.7862.

Compared with the untuned model, which had 0.8591, 0.8693, 0.8692 for training, 0.7838, 0.7755, 0.7586 for testing, with training size equal to 100, 200 and 300 respectively, the final F1 score is slightly lower on the training set, but better on the test set.

I consider the tuning to have worked, because essentially what we care about is whether or not the model can generalize well on the test set, and since the final model outperforms the default ones on the test set, this is a good result.

One sidenote I should make is that, I only tuned C and gamma parameters for SVM with a rbf kernel. Using grid search or some other searching algorithms in the parameter space including different kernels, may give better results.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.